In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
images = sorted(glob.glob("/kaggle/input/lego-minifigure-faces/*/*"))

In [ ]:
tryimage = plt.imread(images[0])
plt.imshow(tryimage);
plt.axis("off");

In [ ]:
plt.figure(figsize=(20,20))
for i in np.arange(10):
    plt.subplot(1,10,i+1)
    plt.imshow(plt.imread(images[i]))
    plt.axis("off")
plt.figure(figsize=(20,20));
for i in np.arange(10,20):
    plt.subplot(2,10,i+1)
    plt.imshow(plt.imread(images[i]));
    plt.axis("off")
plt.figure(figsize=(20,20));
for i in np.arange(5):
    plt.subplot(1,5,i+1)
    plt.imshow(plt.imread(images[20+i]));
    plt.axis("off")

In [ ]:
import cv2
gray_images = []
for i in images:
    gray_images.append(cv2.cvtColor(plt.imread(i), cv2.COLOR_BGR2GRAY))

In [ ]:
images = np.array(gray_images)

In [ ]:
images.shape

In [ ]:
plt.figure(figsize=(20,20))
for i in np.arange(10):
    plt.subplot(1,10,i+1)
    plt.imshow(images[i])
    plt.axis("off")
plt.figure(figsize=(20,20));
for i in np.arange(10,20):
    plt.subplot(2,10,i+1)
    plt.imshow(images[i]);
    plt.axis("off")
plt.figure(figsize=(20,20));
for i in np.arange(5):
    plt.subplot(1,5,i+1)
    plt.imshow(images[20+i]);
    plt.axis("off")

In [ ]:
from keras.models import Model
from keras.layers import Dense,Input

In [ ]:
x_train = [] 
for i in np.arange(15):
    x_train.append(images[i])

In [ ]:
x_test = []
for i in np.arange(15,25):
    x_test.append(images[i])

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0


In [ ]:
x_train[0]

In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam,Adagrad
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy

In [ ]:
input_img = Input(shape = (1024,))

#Model
#decoded daki dense sayısı ile encoded_input değerindeki Input değeri eşit olmalı(?)
#decoder_layer burada decoded'daki giriş boyutu kadar giriş boyutu alır(?)

encoded = Dense(300,activation="tanh")(input_img)
encoded = Dense(512,activation="tanh")(encoded)
encoded = Dense(450,activation="tanh")(encoded)
encoded = Dense(300,activation="tanh")(encoded)
decoded = Dense(512,activation="sigmoid")(encoded)
decoded = Dense(450,activation="sigmoid")(decoded)
decoded = Dense(300,activation="sigmoid")(decoded)
decoded = Dense(1024,activation="sigmoid")(decoded)

autoencoder = Model(input_img,decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.0001),
                   loss=BinaryCrossentropy())
hist = autoencoder.fit(x_train,x_train,epochs=400,
                      batch_size=4,shuffle=True,
                       validation_data=(x_test,x_test))

In [ ]:
x_train.shape

In [ ]:
encoder_model = Model(input_img,encoded)
encoded_input = Input(shape=(1024,300,))
decoder_layer = autoencoder.layers[-1]
decoder_model = Model(encoded_input,decoder_layer(encoded_input))

encoded_imgs = encoder_model.predict(x_train)
#decoded hata veriyor
decoded_imgs = decoder_model.predict(encoded_imgs)

In [ ]:
aa = autoencoder.predict(x_test)

In [ ]:
plt.imshow(aa[0]);

In [ ]:
decoder_model.summary()

In [ ]:
plt.imshow(decoded_imgs[0]);

In [ ]:
for i in hist.history:
    plt.plot(hist.history[i],label=i);
    plt.legend()